This file is based on finding $\beta$ from the data consisting of refractive index and wavelength for a waveguide strcuture.

In [2]:
import numpy as np
import pandas as pd

Assuming the csv file contains 2 columns: wl and eta<br>
wl = wavelength (in metres), eta = refractive index

In [ ]:
d1 = pd.read_csv('');
d2 = pd.read_csv('');
d3 = pd.read_csv('');

In [ ]:
c = 3000000000

def freq(wl):
    return c/wl

In [ ]:
d1['freq'] = freq(d1['wl'])
d2['freq'] = freq(d1['wl'])
d3['freq'] = freq(d1['wl'])

In [ ]:
sel = ['freq','eta']
data1 = np.concatenate((d1[sel].values.astype(np.float64),d2[sel].values.astype(np.float64),d3[sel].values.astype(np.float64)), axis = 0)

I have given both Cauchy distribution and Polynomial Regression (of degree 4 (degree can be changed as well)).<br>
Comment out the code which is not required.

In [ ]:
##Cauchy
import scipy.stats as stats
params = stats.cauchy.fit(data1)
x0, gamma = params

def ri(fq):
    return stats.cauchy.pdf(fq, loc=x0, scale=gamma)

In [ ]:
##Polynomial Regression
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures  
poly_regs= PolynomialFeatures(degree= 4)  #can change the degree here
x_poly= poly_regs.fit_transform(data1[:,0].reshape(-1,1))  
lin_reg =LinearRegression()  
lin_reg.fit(x_poly, data1[:,1])  

def ri(fq):
    return lin_reg.predict(poly_regs.fit_transform([[fq]]))

In [ ]:
a = d3['wl'].max()
b = d3['wl'].min()

c = np.arange(a,b+0.5e-09,0.5e-09)
neu_i = freq(c)
eta_i = ri(neu_i)
neu_p = d1['freq']

In [ ]:
for p in range(len(neu_p)):
    neu_s = neu_p[p] - neu_i
    eta_s = ri(neu_s)
    beta = neu_p[p]*ri(neu_p[p]) - neu_i*eta_i - neu_s*eta_s
    df = pd.dataframe(neu_i, columns = ['neu_i'])
    df['eta_i'] = pd.dataframe(eta_i)
    df['neu_s'] = pd.dataframe(neu_s)
    df['eta_s'] = pd.dataframe(eta_s)
    df['beta'] = pd.dataframe(beta)
    df.to_csv(str(neu_p[p]))